In [31]:
from pymongo import MongoClient
from functools import reduce

In [2]:
client = MongoClient("mongodb://localhost:27017")
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [4]:
db = client.db_test

# 1) Laureates outnumber prizes

In [8]:
laureates_number = db.laureates.count_documents({})
prizes_number = db.prizes.count_documents({})
(laureates_number,prizes_number)
# laureates>prizes

(934, 590)

# 2)

In [10]:
# Save a list of names of the databases managed by client
db_names = client.list_database_names()
print(db_names)

# Save a list of names of the collections managed by the "nobel" database
nobel_coll_names = client.db_test.list_collection_names()
print(nobel_coll_names)

['admin', 'config', 'db_test', 'local']
['laureates', 'prizes']


# 3)

In [44]:
# Retrieve sample prize and laureate documents
prize = db.prizes.find_one()
laureate = db.laureates.find_one()

In [46]:
# Get all fields present in each type of document

def find_fields(doc):
    fields = doc.keys()
    all_fields = []
    for field in fields:
        if isinstance(doc.get(field),dict):
            all_fields.append((field,doc.get(field)))
        if isinstance(doc.get(field),list):
            subfields = ([list(f.keys()) for f in doc.get(field)])
            subfields = set(reduce(lambda a, b: a+b, subfields))
            all_fields.append((field,subfields))
        else:
            all_fields.append(field)
    return all_fields

In [47]:
find_fields(prize)

['_id',
 'year',
 'category',
 ('laureates', {'firstname', 'id', 'motivation', 'share', 'surname'})]

In [48]:
find_fields(laureate)

['_id',
 'id',
 'firstname',
 'surname',
 'born',
 'died',
 'bornCountry',
 'bornCountryCode',
 'bornCity',
 'gender',
 ('prizes', {'affiliations', 'category', 'motivation', 'share', 'year'})]

# 4)

In [52]:
before_1800 = db.laureates.count_documents({"born": {"$lt": "1800"}})
before_1700 = db.laureates.count_documents({"born": {"$lt": "1700"}})
(before_1800,before_1700)

(38, 38)

# 5)

In [54]:
criteria1 = {'bornCountry': 'Germany'}
criteria2 = {'bornCountry': 'Germany', 
            'diedCountry': 'USA'}
criteria3 = {'bornCountry': 'Germany', 
            'diedCountry': 'USA', 
            'firstname': 'Albert'}
criteria_list = [criteria1,criteria2,criteria3]

In [55]:
[db.laureates.count_documents(criteria) for criteria in criteria_list]

[63, 5, 1]

# 6)

In [56]:
# Save a filter for laureates born in the USA, Canada, or Mexico
criteria = { 'bornCountry': 
                { "$in": ['USA','Canada','Mexico']}
             }

# Count them and save the count
count = db.laureates.count_documents(criteria)
print(count)

291


In [57]:
# Save a filter for laureates who died in the USA and were not born there
criteria = { 'diedCountry': 'USA',
               'bornCountry': { "$ne": 'USA'}, 
             }

# Count them
count = db.laureates.count_documents(criteria)
print(count)

69


# 7) dot notation and the $ne operator

# 8) 

In [58]:
# Filter for laureates born in Austria with non-Austria prize affiliation
criteria = {'bornCountry': 'Austria', 
              'prizes.affiliations.country': {"$ne": 'Austria'}}

# Count the number of such laureates
count = db.laureates.count_documents(criteria)
print(count)

10


# 9)

In [61]:
criteria = {'born':{'$exists':False}}
db.laureates.count_documents(criteria)

0

In [59]:
# Filter for laureates with at least three prizes
criteria = {"prizes.2": {'$exists': True}}

# Find one laureate with at least three prizes
doc = db.laureates.find_one(criteria)

# Print the document
print(doc)

{'_id': ObjectId('65550468d1fc86246b056ee7'), 'id': '482', 'firstname': 'Comité international de la Croix Rouge (International Committee of the Red Cross)', 'born': '0000-00-00', 'died': '0000-00-00', 'gender': 'org', 'prizes': [{'year': '1917', 'category': 'peace', 'share': '1', 'affiliations': [[]]}, {'year': '1944', 'category': 'peace', 'share': '1', 'affiliations': [[]]}, {'year': '1963', 'category': 'peace', 'share': '2', 'affiliations': [[]]}]}
